In [269]:
import pandas as pd
import json
import requests
import folium

In [270]:
incidentsdf = pd.read_csv('incidents_may.csv', usecols=["id", "priority", "implicated", "incident_time", "latitude", "longitude"])

#drop duplicates
incidentsdf = incidentsdf.drop_duplicates(subset="id", keep="first")

#cast incident_time to datetime
incidentsdf["incident_time"] = pd.to_datetime(incidentsdf["incident_time"])

#fill nan to 0 and cast implicated to int
incidentsdf["implicated"] = incidentsdf["implicated"].fillna(0)
incidentsdf["implicated"] = incidentsdf["implicated"].astype(int)

#calculate intervals and create column
new = incidentsdf.groupby(pd.Grouper(key="incident_time", freq='60Min')).apply(lambda x: x['incident_time'])
incidentsdf["interval"] = new.index.get_level_values(0)

#multiindex by interval, then id
incidentsdf = incidentsdf.set_index(["interval", "id"])

#print head
incidentsdf.head()

priority  latitude  longitude  implicated  \
interval            id                                                 
2022-04-30 23:00:00 57518         2  4.705279 -74.127686           2   
                    57519         1  4.600571 -74.143379           3   
2022-05-01 00:00:00 57520         1  4.647777 -74.137123           1   
                    57521         1  4.720405 -74.075302           2   
                    57522         1  4.614258 -74.167252           2   

                                incident_time  
interval            id                         
2022-04-30 23:00:00 57518 2022-04-30 23:59:00  
                    57519 2022-04-30 23:54:00  
2022-05-01 00:00:00 57520 2022-05-01 00:08:00  
                    57521 2022-05-01 00:10:00  
                    57522 2022-05-01 00:11:00

In [271]:
agentsdf = pd.read_csv('agents_may.csv', usecols=["dev_id", "time_stamp", "latitude", "longitude"])

#cast time_stamp to datetime
agentsdf["time_stamp"] = pd.to_datetime(agentsdf["time_stamp"])

#calculate intervals and create column
new = agentsdf.groupby(pd.Grouper(key="time_stamp", freq='60Min')).apply(lambda x: x['time_stamp'])
agentsdf["interval"] = new.index.get_level_values(0)

#multiindex by interval, then id
agentsdf = agentsdf.set_index(["interval", "dev_id"])

#print head
agentsdf.head()

time_stamp  latitude  longitude
interval   dev_id                                                  
2022-05-01 868033050089715 2022-05-01 00:01:40  4.551646 -74.108515
           868033050102534 2022-05-01 00:01:42  4.650030 -74.147322
           868033050090085 2022-05-01 00:01:47  4.581688 -74.128348
           868033050103508 2022-05-01 00:01:52  4.551098 -74.147635
           868033050101650 2022-05-01 00:01:55  4.619840 -74.159668

In [272]:
agentsdf.groupby(pd.Grouper(key="time_stamp", freq='5Min'))

In [273]:
incidents_locations = incidentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
incidents_locations.head()

interval             id   
2022-04-30 23:00:00  57518       -74.127685546875,4.705278873443604
                     57519    -74.14337921142578,4.6005706787109375
2022-05-01 00:00:00  57520    -74.13712310791016,4.6477766036987305
                     57521     -74.07530212402344,4.720405101776123
                     57522     -74.16725158691406,4.614258289337158
dtype: object

In [274]:
agents_locations = agentsdf[["longitude","latitude"]].apply(lambda x: ','.join(x.values.astype(str)), axis=1)
agents_locations.head()

interval    dev_id         
2022-05-01  868033050089715     -74.108515,4.5516463
            868033050102534    -74.1473218,4.6500299
            868033050090085    -74.1283484,4.5816876
            868033050103508    -74.1476347,4.5510981
            868033050101650      -74.1596683,4.61984
dtype: object

In [275]:
incidents_sample = incidents_locations.sample(100)
agents_sample = agents_locations.sample(100)

In [276]:
folium_map = folium.Map(
    location=[4.6534649, -74.0836453], zoom_start=12, tiles="CartoDB positron"
)

incidents_feature_group = folium.FeatureGroup(name="Incidents")
for i in range(len(incidents_sample)):
    marker = folium.CircleMarker(
        location=incidents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="red",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(incidents_feature_group)

agents_feature_group = folium.FeatureGroup(name="Agents")
for i in range(len(agents_sample)):
    marker = folium.CircleMarker(
        location=agents_sample.iloc[i].split(",")[::-1],
        radius=1,
        color="blue",
        fill=True,
        fill_opacity=0.3,
    )
    marker.add_to(agents_feature_group)

incidents_feature_group.add_to(folium_map)
agents_feature_group.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map


In [277]:
agents_coordinates = ";".join(agents_sample)
incidents_coordinates = ";".join(incidents_sample)
coordinates = agents_coordinates + ";" + incidents_coordinates

agents_count = agents_sample.shape[0]
sources = ";".join(str(i) for i in range(agents_count))

incidents_count = incidents_sample.shape[0]
destinations = ";".join(
    str(i) for i in range(agents_count, agents_count + incidents_count)
)

#Ojo con correr esto sin estar cirriendo el servicio.
durations = json.loads(
    requests.get(
        f"http://127.0.0.1:5000/table/v1/driving/{coordinates}?sources={sources}&destinations={destinations}"
    ).text
)["durations"]


In [278]:
#Finally, we can create a dataframe with the durations in seconds.}

df = pd.DataFrame(durations)
#df = df.apply(lambda x: x/60)

df = df.set_index(agents_sample.index.get_level_values(1))
df = df.set_axis(list(incidents_sample.index.get_level_values(1)), axis=1)

df


,59020,57836,66314,63474,66150,60841,64599,64055,66366,63631,...,60836,62969,58624,66734,60615,57603,61222,62856,60504,64643
dev_id,,,,,,,,,,,,,,,,,,,,,
868033050140518,868.2,1642.4,1045.6,865.4,2026.6,991.5,1042.2,1302.0,658.9,868.2,...,960.7,1224.6,946.2,699.4,1539.8,1679.9,2062.1,1433.7,1301.8,330.4
868033050103953,737.4,1162.3,565.5,987.6,1625.5,511.4,530.2,1070.7,1538.7,988.4,...,856.7,825.7,686.3,926.0,1059.7,1015.8,1729.4,785.3,2223.3,1249.6
868033050139528,991.7,600.7,859.3,1143.4,1063.9,805.2,892.7,683.5,1870.3,1146.1,...,1100.2,882.0,1007.6,1420.5,392.7,502.1,1167.8,702.1,2514.1,1477.4
868033050089566,1177.9,260.8,1045.5,1256.7,412.1,991.4,1078.9,740.2,2056.5,1259.4,...,1286.4,1068.2,1193.8,1606.7,581.6,818.8,516.0,898.0,2700.3,1663.6
868033050089699,830.7,351.6,698.3,916.9,790.1,644.2,731.7,291.8,1709.3,919.6,...,939.2,677.2,846.6,1259.5,627.3,781.0,894.0,860.2,2353.1,1316.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868033050102815,1004.3,1566.9,970.1,1001.5,2030.1,916.0,966.7,1475.3,945.6,1004.3,...,935.2,1230.3,870.7,385.5,1464.3,1583.3,2134.0,1352.8,1630.2,676.3
868033050089541,698.7,335.7,566.3,850.4,798.9,512.2,599.7,390.5,1577.3,853.1,...,807.2,589.0,714.6,1127.5,233.1,373.2,902.8,452.4,2221.1,1184.4
868033050103243,276.0,896.8,526.2,498.2,1258.5,472.1,351.4,487.2,1381.0,500.9,...,395.3,172.6,600.1,1013.9,804.6,944.7,1294.0,824.6,2023.9,987.2
